# Gather split statistics

Problem: Split statistics are held in individual txt-files. 
To make use of the more easily, it would be convenient to gather all this information into a single csv file.

In [1]:
import pathlib
import re
import sys
sys.path.append(str(pathlib.Path("__file__").absolute().parents[1]))

import pandas as pd

from src.util.definitions import DATA_ROOT

In [2]:
# read all the statistics files
rows = []
for split in (DATA_ROOT / "splits").iterdir():
    match_split = re.search(r"synferm_dataset_2023-12-20_(\dD)_split_*(.*)", split.name)
    match_split_synthetic = re.search(r"synferm_dataset_2024-01-31_synthetic_(\dD)_split_*(.*)", split.name)
    if split.is_dir():
        if match_split:
            split_name = "_".join(match_split.groups()).strip("_")
            for file in split.iterdir():
                match = re.search(r"fold(\d+)_statistics\.txt", file.name)
                if match:
                    with open(file, "r") as f:
                        content = {"split_name": split_name, "fold": match.group(1)}
                        content.update({k:v  for k,v in [line.strip("\n").split(": ") for line in f.readlines()]})
                        rows.append(content)                
        elif match_split_synthetic:
            split_name = "_".join(match_split_synthetic.groups()).strip("_")
            for file in split.iterdir():
                match = re.search(r"fold(\d+)_statistics\.txt", file.name)
                if match:
                    with open(file, "r") as f:
                        content = {"split_name": f"{split_name}_syn", "fold": match.group(1)}
                        content.update({k:v  for k,v in [line.strip("\n").split(": ") for line in f.readlines()]})
                        rows.append(content)  

In [3]:
# create df to hold all info
df = pd.DataFrame(rows).sort_values(by=["split_name", "fold"], ignore_index=True)
df

,split_name,fold,Train samples,Val samples,Test samples,Train samples binary_A has label 1,Train samples binary_B has label 1,Train samples binary_C has label 1,Val samples binary_A has label 1,Val samples binary_B has label 1,...,Train initiators,Val initiators,Test initiators,Train monomers,Val monomers,Test monomers,Train terminators,Val terminators,Test terminators,Not used
0,0D_0.625,0,246 (0.6%),19743 (50.0%),19497 (49.4%),195 (79.3%),123 (50.0%),67 (27.2%),16267 (82.4%),11439 (57.9%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0D_0.625,1,246 (0.6%),19743 (50.0%),19497 (49.4%),197 (80.1%),145 (58.9%),77 (31.3%),16227 (82.2%),11458 (58.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0D_0.625,2,246 (0.6%),19743 (50.0%),19497 (49.4%),203 (82.5%),148 (60.2%),78 (31.7%),16201 (82.1%),11430 (57.9%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0D_0.625,3,246 (0.6%),19743 (50.0%),19497 (49.4%),197 (80.1%),141 (57.3%),65 (26.4%),16214 (82.1%),11456 (58.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0D_0.625,4,246 (0.6%),19743 (50.0%),19497 (49.4%),204 (82.9%),120 (48.8%),56 (22.8%),16223 (82.2%),11448 (58.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,3D_80,4,19544 (49.5%),43 (0.1%),26 (0.1%),16373 (83.8%),11539 (59.0%),5475 (28.0%),29 (67.4%),17 (39.5%),...,53,7,4,48,7,8,32,4,4,19873 (50.3%)
357,3D_80,5,19795 (50.1%),38 (0.1%),39 (0.1%),16226 (82.0%),10824 (54.7%),5663 (28.6%),35 (92.1%),16 (42.1%),...,53,7,7,48,7,8,32,4,5,19614 (49.7%)
358,3D_80,6,18709 (47.4%),48 (0.1%),68 (0.2%),15472 (82.7%),10854 (58.0%),4864 (26.0%),44 (91.7%),37 (77.1%),...,53,7,7,47,6,9,32,4,5,20661 (52.3%)
359,3D_80,7,19819 (50.2%),49 (0.1%),46 (0.1%),15976 (80.6%),11722 (59.1%),6187 (31.2%),47 (95.9%),29 (59.2%),...,53,6,7,48,7,7,32,4,5,19572 (49.6%)


In [8]:
# save to file for later use
df.to_csv(DATA_ROOT / "splits" / "split_statistics_2023-12-20.csv", index=False)

In [11]:
df.loc[df["split_name"] == "0D_80"]

,split_name,fold,Train samples,Val samples,Test samples,Train samples binary_A has label 1,Train samples binary_B has label 1,Train samples binary_C has label 1,Val samples binary_A has label 1,Val samples binary_B has label 1,...,Train initiators,Val initiators,Test initiators,Train monomers,Val monomers,Test monomers,Train terminators,Val terminators,Test terminators,Not used
235,2D_80,0,24829 (62.9%),380 (1.0%),443 (1.1%),21110 (85.0%),14989 (60.4%),7529 (30.3%),226 (59.5%),151 (39.7%),...,53,6,7,48,7,7,41,40,41,13834 (35.0%)
236,2D_80,1,23898 (60.5%),372 (0.9%),586 (1.5%),19554 (81.8%),13897 (58.2%),6861 (28.7%),291 (78.2%),207 (55.6%),...,53,7,7,47,7,9,41,40,41,14630 (37.1%)
237,2D_80,2,24652 (62.4%),524 (1.3%),350 (0.9%),21199 (86.0%),14546 (59.0%),7847 (31.8%),415 (79.2%),377 (71.9%),...,53,7,7,48,6,7,41,41,41,13960 (35.4%)
238,2D_80,3,25144 (63.7%),355 (0.9%),382 (1.0%),20247 (80.5%),14125 (56.2%),6953 (27.7%),290 (81.7%),205 (57.7%),...,53,7,7,48,7,7,41,38,41,13605 (34.5%)
239,2D_80,4,24451 (61.9%),532 (1.3%),370 (0.9%),20146 (82.4%),14119 (57.7%),7235 (29.6%),508 (95.5%),388 (72.9%),...,53,7,7,48,7,7,41,40,40,14133 (35.8%)
240,2D_80,5,24307 (61.6%),569 (1.4%),374 (0.9%),20351 (83.7%),14296 (58.8%),7044 (29.0%),427 (75.0%),321 (56.4%),...,53,7,7,48,7,8,41,41,41,14236 (36.1%)
241,2D_80,6,24394 (61.8%),510 (1.3%),436 (1.1%),20039 (82.1%),14278 (58.5%),7141 (29.3%),400 (78.4%),271 (53.1%),...,53,7,7,48,7,7,41,41,40,14146 (35.8%)
242,2D_80,7,24584 (62.3%),428 (1.1%),354 (0.9%),20633 (83.9%),14565 (59.2%),7459 (30.3%),330 (77.1%),228 (53.3%),...,53,7,7,48,7,7,41,41,41,14120 (35.8%)
243,2D_80,8,26121 (66.2%),315 (0.8%),427 (1.1%),21579 (82.6%),15189 (58.1%),7679 (29.4%),222 (70.5%),155 (49.2%),...,53,7,7,48,7,8,41,40,41,12623 (32.0%)
